In [17]:
import requests
import os
import numpy as np
import cv2
import json
import datetime

results = []
BASE_URL = "http://localhost:8000/api" 
ENDPOINTS = [
    {"path": "/detect/", "method": "POST", "allowed": ["DIRECTOR", "MANAGER", "STAFF"]},
    # DIRECTOR 전용
    {"path": "/company/members/", "method": "GET", "allowed": ["DIRECTOR"]},
    # DIRECTOR, MANAGER 전용
    {"path": "/projects/", "method": "DELETE", "allowed": ["DIRECTOR", "MANAGER"]}
]


In [18]:
import requests
import os
import numpy as np
import cv2
import json
import datetime

# 1. 초기 설정 및 리스트 선언 (초기화 방지)
BASE_URL = "http://localhost:8000/api"
PASSWORD = "password123"
results = []  # 테스트 결과가 저장될 공간

TEST_SCENARIOS = {
    "DIRECTOR": {"email": "testdirector@naver.com", "p_id": 301},
    "MANAGER":  {"email": "testmanager@naver.com", "p_id": 302},
    "STAFF":    {"email": "teststaff@naver.com", "p_id": 303}
}

def get_real_dummy_image():
    img = np.zeros((100, 100, 3), dtype=np.uint8)
    _, img_encoded = cv2.imencode('.jpg', img)
    return img_encoded.tobytes()

# 2. 테스트 수행 및 데이터 수집 (results.append가 핵심)
for role, data in TEST_SCENARIOS.items():
    print(f"--- [{role}] 검증 중... ---")
    session = requests.Session()
    
    # 로그인
    login_res = session.post(f"{BASE_URL}/login/", data={"email": data["email"], "password": PASSWORD})
    csrf_token = session.cookies.get('csrftoken')
    headers = {'X-CSRFToken': csrf_token}

    if login_res.status_code == 200:
        # [A] 회원 관리
        path_member = "/company/members/"
        res_m = session.get(f"{BASE_URL}{path_member}", headers=headers)
        results.append({"path": path_member, "text": f"--- [{role}] --- > 회원 관리 접속: {res_m.status_code}"})

        # [B] 프로젝트 삭제 (params='id' 사용)
        path_project = "/projects/"
        res_p = session.delete(f"{BASE_URL}{path_project}", params={"id": data["p_id"]}, headers=headers)
        results.append({"path": path_project, "text": f"--- [{role}] --- > 프로젝트 삭제 (ID:{data['p_id']}): {res_p.status_code}"})

        # [C] 결함 탐지
        path_detect = "/detect/"
        files = {'image': ('test.jpg', get_real_dummy_image(), 'image/jpeg')}
        res_d = session.post(f"{BASE_URL}{path_detect}", files=files, data={'project_id': str(data["p_id"])}, headers=headers)
        results.append({"path": path_detect, "text": f"--- [{role}] --- > 결함 탐지 수행: {res_d.status_code}"})
    else:
        print(f" ! [{role}] 로그인 실패")

--- [DIRECTOR] 검증 중... ---
--- [MANAGER] 검증 중... ---
--- [STAFF] 검증 중... ---


In [19]:
# 3. 결과 저장 (qa/model_qa_results/auth_test_report.txt)
save_dir = "model_qa_results"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

file_path = os.path.join(save_dir, "auth_test_report.txt")

with open(file_path, 'w', encoding='utf-8') as f:
    f.write("=== [QA] 역할별 권한 분리 테스트 리포트 ===\n")
    f.write(f"테스트 완료 시간: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    
    if not results:
        f.write("\n⚠️ 테스트 결과 데이터가 수집되지 않았습니다.\n")
    else:
        current_path = ""
        for item in results:
            if current_path != item['path']:
                current_path = item['path']
                f.write(f"\n[API 경로: {current_path}]\n")
                f.write("-" * 50 + "\n")
            f.write(item['text'] + "\n")
            
    f.write("\n=== 테스트 완료 ===\n")

print(f"\n리포트 저장 완료: {os.path.abspath(file_path)}")


리포트 저장 완료: D:\final_project\qa\model_qa_results\auth_test_report.txt
